Attemping to plot a Hertzsprung-Russell diagram

In [1]:
!pip install PyAstronomy
!pip install plotly

In [2]:
import pandas as pd
from PyAstronomy import pyasl
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [3]:
filepath = 'C:\Py\hygdata_v3.csv' #HYG Star Database

In [4]:
starData_full = pd.read_csv(filepath)

In [5]:
starData = starData_full[0:1000] #Taking a smaller number of entries for performance

In [8]:
starData.keys()

Index(['id', 'hip', 'hd', 'hr', 'gl', 'bf', 'proper', 'ra', 'dec', 'dist',
       'pmra', 'pmdec', 'rv', 'mag', 'absmag', 'spect', 'ci', 'x', 'y', 'z',
       'vx', 'vy', 'vz', 'rarad', 'decrad', 'pmrarad', 'pmdecrad', 'bayer',
       'flam', 'con', 'comp', 'comp_primary', 'base', 'lum', 'var', 'var_min',
       'var_max'],
      dtype='object')

In [9]:
starData['proper']

0      Sol
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
995    NaN
996    NaN
997    NaN
998    NaN
999    NaN
Name: proper, Length: 1000, dtype: object

In [10]:
def proper_name_finder(list_of_stars):
    for i in list_of_stars['proper']:
        if type(i) == str:
            print(i)
def index_finder(list_of_stars, proper_name):
    return list(list_of_stars['proper']).index(proper_name)

proper_name_finder(starData)
index_finder(starData, 'Alpheratz')

Sol
Alpheratz
Caph


676

In [11]:
def ktemp(ci): #Takes a color index (Blue magnitude minus visual magnitude) and returns a temperature in Kelvin
    kelvin = 4600 * ((.92*(ci) + 1.7)**-1 + 
        (0.92*(ci) + .62)**-1)
    return kelvin
b = pyasl.BallesterosBV_T() #The same function as ktemp but imported from PyAstronomy's library

In [12]:
ktemp(starData['ci'][0]) #

5756.588112693915

In [13]:
b.bv2T(starData['ci'][0]) # Google says 5,778 K
# Ballesteros' forumula is derived from the black body approximation 
# which can be off by 30-40k according to PyAstronomy documentation

5756.588112693915

In [14]:
color_index_list = list(map(lambda x: x, starData['ci']))
abs_mag_list = list(map(lambda x: x, starData['absmag']))

In [15]:
layout = {'title': 'Hertzsprung-Russell of HYG', 
          'xaxis': {'title': 'B-V'},
          'yaxis': {'title': 'Absolute Mag'}
         }
trace = {'x': color_index_list, 'y': abs_mag_list, 'mode': 'markers','marker': {'color': 'red', 'size': 2}}

figure = {'data': [trace], 'layout': layout}

plotly.offline.iplot(figure)

In [16]:
temp_list = list(map(lambda x: ktemp(x), color_index_list))

In [17]:
temp_list[0]

5756.588112693915

In [18]:
layout2 = {'title': 'Hertzsprung-Russell of HYG', 
          'xaxis': {'title': 'Temperature (K)'},
          'yaxis': {'title': 'Absolute Mag'}
         }
trace2 = {'x': temp_list, 
          'y': abs_mag_list, 
          'mode': 'markers',
          'marker': {'color': 'red', 'size': 2}}

figure2 = {'data': [trace2], 
           'layout': layout2}

plotly.offline.iplot(figure2)

In [19]:
starData['lum'][0] # star luminosity in units of solar output

1.0

In [20]:
lum_list = list(map(lambda x: x, starData['lum']))

In [21]:
layout3 = {'title': 'Hertzsprung-Russell plot of stars in the HYG database', 
          'xaxis': {'title': 'Temperature (K)'},
          'yaxis': {'title': 'Luminosity', 'type': 'log'}
         }
trace3 = {'x': temp_list, 
          'y': lum_list, 
          'mode': 'markers',
          'marker': {'color': 'red', 'size': 2}}

figure3 = {'data': [trace3], 
           'layout': layout3}


plotly.offline.iplot(figure3)

Planck's Law

In [22]:
h = 6.6260700e-34 # m^2 kg s^-1 
c = 299792458     # m s^-1
e = 2.7182818
k = 1.3806485e-23 # m^2 kg s^-2 K^-1
def micro(wavelength):               # Converts meters to micrometers
    return (wavelength * 1e-6)

In [23]:
def planck(temperature, wavelength): # This function accepts temperature in Kelvin and wavelength in micrometers, and returns 
                                     # the spectral radiance in units of W sr^-1 m^-2
    lam = micro(wavelength)
    expr1 = (2 * h * c**2) / (lam**5)
    expr2 = (h * c) / (k * lam * temperature)
    return expr1 * (e**expr2 - 1)**-1

In [24]:
def easyunits(t, lam):     # This function converts planck() into more confusing but manageable units, kW sr^-1 m^-2 nm^-1
    return planck(t, lam)*1e-9/1000

In [25]:
x_values = list(map(lambda x: round(0.01 + .01* x, 3), list(range(0,300))))
y_values = list(map(lambda x: easyunits(5756, x), x_values))
trace = {'x': x_values, 
         'y': y_values}

In [26]:
plotly.offline.iplot([trace])

In [27]:
filepath2 = 'C:\Py\AM0AM1_5.xls'
sun_Spectrum_full = pd.read_excel(filepath2)
sun_Spectrum_full.keys()

Index(['ASTM G173-03 Reference Spectra Derived from SMARTS v. 2.9.2 (AM1.5)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'ASTM E-490 AM0 Standard Spectra', 'Unnamed: 6'],
      dtype='object')

In [28]:
wavelength = list(map(lambda x: x,sun_Spectrum_full['ASTM G173-03 Reference Spectra Derived from SMARTS v. 2.9.2 (AM1.5)']))
irradiance = list(map(lambda x: x, sun_Spectrum_full['Unnamed: 1']))

In [29]:
wavelength = wavelength[1:2003]
irradiance = irradiance[1:2003]
wavelength_um = list(map(lambda x: x / 1e3, wavelength))         # Converts from nm to um
irradiance_sr = list(map(lambda x: x * 4 * 3.14159, irradiance)) # Adds steradians to the units


In [30]:
solar_irradiance_trace = {'x': wavelength_um, 
                          'y': irradiance_sr}
planck_law_list = list(map(lambda x: easyunits(5756, x), wavelength_um))
planck_law_trace = {'x': wavelength_um, 
                    'y': planck_law_list}

plotly.offline.iplot([solar_irradiance_trace, planck_law_trace])

In [31]:
error = list(map(lambda x, y: x - y, planck_law_list, irradiance_sr)) # total error
residual_sum_of_squares = sum(list(map(lambda x: x**2, error)))       
root_mean_square_error = (residual_sum_of_squares / len(error))**.5
normalized = root_mean_square_error / (max(planck_law_list)-min(planck_law_list))

In [32]:
normalized # normalized RMSE

0.09616987288315385